# 🐬 ✦˚₊‧⁺˖ <b>llm-finetuning-platform</b> ˖⁺‧₊˚✦


### <b>Part 1: Model setup</b>
This notebook is intended to run on Google Colab A100 GPU.

In [1]:
# packages
!pip install -q transformers accelerate peft bitsandbytes datasets tensorboard scipy anthropic openai
import os
import sys

import bitsandbytes
import datasets
import datetime
from datasets import load_dataset
import huggingface_hub
import json
import random
import torch
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline
import peft

import anthropic
import openai

In [2]:
# get secrets
from google.colab import userdata
ANTHROPIC_API_KEY = userdata.get('anthropicsecretkey')
HF_TOKEN = userdata.get('bab')
OPENAI_API_KEY = userdata.get('openaisecretkey')

# login to huggingface
from huggingface_hub import login
login(new_session=False)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


### Load in Llama

In [3]:
##### from claude helper code
# load model in
model_name = "meta-llama/Llama-3.1-8B" ## originally:  "meta-llama/Meta-Llama-3-8B-Instruct"

print(f"Loading {model_name}... This may take a minute...")

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    dtype=torch.float16,  # fp16 saves memory vs fp32
    device_map="auto",
    trust_remote_code=True,
)
# load in tokens
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

Loading meta-llama/Llama-3.1-8B... This may take a minute...


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

### Create the ModelInterface method to select APIs

In [4]:
class ModelInterface:
    """Unified interface for calling different models from APIs"""

    def __init__(self):
        self.openai_client = openai.OpenAI(api_key=OPENAI_API_KEY) if 'OPENAI_API_KEY' in globals() else None
        self.anthropic_client = anthropic.Anthropic(api_key=ANTHROPIC_API_KEY) if 'ANTHROPIC_API_KEY' in globals() else None
        self.local_model = model if 'model' in globals() else None
        self.local_tokenizer = tokenizer if 'tokenizer' in globals() else None

    def generate(self, prompt, model_name, max_tokens=150, temperature=0.7):
        """Generate response from specified model"""

        if model_name == "gpt-5.2":
            return self._generate_openai(prompt, "gpt-5.2", max_tokens, temperature)

        elif model_name == "gpt-3.5-turbo":
            return self._generate_openai(prompt, "gpt-3.5-turbo", max_tokens, temperature)

        elif model_name == "claude-haiku-4-5-20251001":
            return self._generate_anthropic(prompt, "claude-haiku-4-5-20251001", max_tokens, temperature)

        elif model_name == "claude-opus-4-5-20251101":
            return self._generate_anthropic(prompt, "claude-opus-4-5", max_tokens, temperature)

        elif model_name == "Llama-3.1-8B":
            return self._generate_local(prompt, max_tokens, temperature)

        else:
            raise ValueError(f"Unknown model: {model_name}")

    def _generate_openai(self, prompt, model, max_tokens, temperature):
      """Generate from OpenAI API. Select correct token argument per model."""
      kwargs = {
          "model": model,
          "messages": [{"role": "user", "content": prompt}],
          "temperature": temperature,
          }
      # newer models ("gpt-5") use max_completion_tokens instead of max_tokens
      if model.startswith("gpt-5"):
          kwargs["max_completion_tokens"] = max_tokens
      else:
          kwargs["max_tokens"] = max_tokens
      response = self.openai_client.chat.completions.create(**kwargs)
      return response.choices[0].message.content

    def _generate_anthropic(self, prompt, model, max_tokens, temperature):
        """Generate from Anthropic API."""
        response = self.anthropic_client.messages.create(
            model=model,
            max_tokens=max_tokens,
            temperature=temperature,
            messages=[{"role": "user", "content": prompt}]
        )
        return response.content[0].text

    def _generate_local(self, prompt, max_tokens, temperature):
        """Generate from local Llama model."""
        inputs = self.local_tokenizer(prompt, return_tensors="pt").to("cuda")

        with torch.no_grad():
            outputs = self.local_model.generate(
                **inputs,
                max_new_tokens=max_tokens,
                temperature=temperature,
                do_sample=True,
                top_p=0.9,
                pad_token_id=self.local_tokenizer.eos_token_id,
            )

        response = self.local_tokenizer.decode(outputs[0], skip_special_tokens=True)
        # remove the prompt from the response
        response = response[len(prompt):].strip()
        return response

In [5]:
##### last steps before all systems go! #####
# initialize interface
interface = ModelInterface()

# test prompt to verify models working
test_prompt = "Complete this sentence: The secret chest finally opened, and "
models_list = ["gpt-5.2"]
print(f"--- testing large language models ---\n")
print(f"prompt: {test_prompt} ---")
for model_name in models_list:
    try:
        print(f"\n--- {model_name} ---")
        response = interface.generate(test_prompt, model_name)
        print(f"Response: {response[:100]}...")
    except Exception as e:
        print(f"Error: {e}")


--- testing large language models ---

prompt: Complete this sentence: The secret chest finally opened, and  ---

--- gpt-5.2 ---
Response: The secret chest finally opened, and a warm gust of air spilled out, carrying the scent of cedar and...


# ☁


#### 🩰  𝓅𝓇𝑒𝓉𝓉𝓎 𝑔𝒾𝓇𝓁𝓈

In [6]:
test_prompt = "Complete this sentence: I went into the attic of my grandparents house, and to my suprise I saw "

models_list = ["gpt-5.2", "gpt-3.5-turbo", "Llama-3.1-8B", "claude-haiku-4-5-20251001", "claude-opus-4-5-20251101"]
print(f"--- testing large language models ---\n")
print(f"prompt: {test_prompt} ---")

for model_name in models_list:
    try:
        print(f"\n--- {model_name} ---")
        response = interface.generate(test_prompt, model_name)
        print(f"response: {response[:1000]}...")
    except Exception as e:
        print(f"error: {e}")

print("\n ")

--- testing large language models ---

prompt: Complete this sentence: I went into the attic of my grandparents house, and to my suprise I saw  ---

--- gpt-5.2 ---
response: I went into the attic of my grandparents' house, and to my surprise I saw **a dusty trunk with my name written on it in faded ink.**...

--- gpt-3.5-turbo ---
response:  a dusty old trunk filled with family heirlooms and photos from generations past....

--- Llama-3.1-8B ---
response: 3 ___________.
A. ghosts
B. skeletons
C. cats
D. toys
E. none of the above
Answer: B...

--- claude-haiku-4-5-20251001 ---
response: # I went into the attic of my grandparents house, and to my surprise I saw

...a trunk filled with old photographs and letters dating back decades, along with my grandmother's wedding dress carefully preserved in tissue paper.

---

Would you like me to suggest other possible endings, or would you like to continue this story?...

--- claude-opus-4-5-20251101 ---
response: I went into the attic of my gra

# Scratch/Testing Area

In [7]:
############# scratch/testing #############
##### trying to get notebook to display in github. trying to delete metadata. having problems with the file path for this notebook...? 😒
import nbformat
import os
from google.colab import drive

# Mount Google Drive to access files from your Drive
#drive.mount('/content/drive')
notebook_to_process = "/content/drive/llm_finetuning_platform_1_1_setup.ipynb"
output_notebook_name = "llm_finetuning_platform_1_1_setup.ipynb"

if os.path.exists(notebook_to_process):
    with open(notebook_to_process, "r", encoding="utf-8") as f:
        nb = nbformat.read(f, as_version=4)

    nb.metadata.pop("widgets", None)

    with open(output_notebook_name, "w", encoding="utf-8") as f:
        nbformat.write(nb, f)

    print(f"Widget metadata removed from {notebook_to_process} and saved to {output_notebook_name}.")
else:
    print(f"Error: The file '{notebook_to_process}' was not found. Please ensure the path is correct and Google Drive is mounted.")

Error: The file '/content/drive/llm_finetuning_platform_1_1_setup.ipynb' was not found. Please ensure the path is correct and Google Drive is mounted.


In [8]:
############# scratch/testing #############
from openai import OpenAI
from anthropic import Anthropic


client = OpenAI(api_key=OPENAI_API_KEY)
client1 = Anthropic(api_key=ANTHROPIC_API_KEY)

#client.models.retrieve("gpt-5.2")

client1.models.list()
#     id='claude-opus-4-5-20251101'
#     display_name='Claude Opus 4.5'

SyncPage[ModelInfo](data=[ModelInfo(id='claude-opus-4-5-20251101', created_at=datetime.datetime(2025, 11, 24, 0, 0, tzinfo=datetime.timezone.utc), display_name='Claude Opus 4.5', type='model'), ModelInfo(id='claude-haiku-4-5-20251001', created_at=datetime.datetime(2025, 10, 15, 0, 0, tzinfo=datetime.timezone.utc), display_name='Claude Haiku 4.5', type='model'), ModelInfo(id='claude-sonnet-4-5-20250929', created_at=datetime.datetime(2025, 9, 29, 0, 0, tzinfo=datetime.timezone.utc), display_name='Claude Sonnet 4.5', type='model'), ModelInfo(id='claude-opus-4-1-20250805', created_at=datetime.datetime(2025, 8, 5, 0, 0, tzinfo=datetime.timezone.utc), display_name='Claude Opus 4.1', type='model'), ModelInfo(id='claude-opus-4-20250514', created_at=datetime.datetime(2025, 5, 22, 0, 0, tzinfo=datetime.timezone.utc), display_name='Claude Opus 4', type='model'), ModelInfo(id='claude-sonnet-4-20250514', created_at=datetime.datetime(2025, 5, 22, 0, 0, tzinfo=datetime.timezone.utc), display_name='Cl

# Save our work!

In [9]:
# document our setup/progress and save in model_setup.json
from datetime import datetime

setup_info = {
    "date": datetime.now().isoformat(),
    "environment": "Google Colab",
    "gpu": torch.cuda.get_device_name(0) if torch.cuda.is_available() else "None",
    "models": {
        "local": ["meta-llama/Llama-3.1-8B"],
        "api": ["gpt-3.5-turbo", "gpt-5.2", "claude-haiku-4-5-20251001", "claude-opus-4-5-20251101"] if 'OPENAI_API_KEY' in globals() else ["None - add API keys"],
    },
    "memory_usage_gb": round(torch.cuda.memory_allocated()/1e9, 2) if torch.cuda.is_available() else 0,
}

with open('model_setup.json', 'w') as f:
    json.dump(setup_info, f, indent=2)